In [1]:
#DATASETS:  https://www.kaggle.com/competitions/daasbstp2023/data

In [23]:
#Let's start by importing all packages needed.
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import seaborn as sns
from sklearn import preprocessing
import numpy as np
import time
from dateutil import parser
import pytz

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree, export_text


from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

from scipy.stats import uniform, poisson

from sklearn.metrics import confusion_matrix, \
                            accuracy_score, \
                            precision_score, \
                            recall_score, \
                            f1_score, \
                            fbeta_score, \
                            mean_squared_error, \
                            mean_absolute_error, \
                            roc_auc_score, \
                            roc_curve, \
                            classification_report


In [3]:
#Load the datasets.
df_en1 = pd.read_csv('~/Downloads/DAA/daasbstp2023/energia_202109-202112.csv', na_filter=False, encoding = "latin")
df_en2 = pd.read_csv('~/Downloads/DAA/daasbstp2023/energia_202201-202212.csv', na_filter=False, encoding = "latin")
df_me1 = pd.read_csv('~/Downloads/DAA/daasbstp2023/meteo_202109-202112.csv', na_filter=False, encoding = "latin")
df_me2 = pd.read_csv('~/Downloads/DAA/daasbstp2023/meteo_202201-202212.csv', na_filter=False, encoding = "latin")

In [4]:
#Append the datasets by type.
df_en = df_en1._append(df_en2,ignore_index=True)
#df_en = df_en._append(df_en3,ignore_index=True)  #test X

df_me = df_me1._append(df_me2,ignore_index=True)
#df_me = df_me._append(df_me3,ignore_index=True)  #test X

In [25]:
df_me.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11688 entries, 0 to 11687
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   dt                   11688 non-null  int64   
 1   dt_iso               11688 non-null  object  
 2   city_name            11688 non-null  category
 3   temp                 11688 non-null  float64 
 4   feels_like           11688 non-null  float64 
 5   temp_min             11688 non-null  float64 
 6   temp_max             11688 non-null  float64 
 7   pressure             11688 non-null  int64   
 8   sea_level            11688 non-null  object  
 9   grnd_level           11688 non-null  object  
 10  humidity             11688 non-null  int64   
 11  wind_speed           11688 non-null  float64 
 12  rain_1h              11688 non-null  object  
 13  clouds_all           11688 non-null  int64   
 14  weather_description  11688 non-null  category
dtypes: category(2), flo

In [6]:
df_en.tail(30)

,Data,Hora,Normal (kWh),Horário Económico (kWh),Autoconsumo (kWh),Injeção na rede (kWh)
10986,2022-12-30,18,1.325,0.000,0.000,None
10987,2022-12-30,19,0.813,0.000,0.000,None
10988,2022-12-30,20,1.072,0.000,0.000,None
10989,2022-12-30,21,0.747,0.000,0.000,None
10990,2022-12-30,22,0.000,0.586,0.000,None
10991,2022-12-30,23,0.000,0.655,0.000,None
10992,2022-12-31,0,0.000,0.718,0.000,None
10993,2022-12-31,1,0.000,0.711,0.000,None
10994,2022-12-31,2,0.000,0.702,0.000,None
10995,2022-12-31,3,0.000,0.700,0.000,None


In [13]:
df_me.head(20)

,dt,dt_iso,city_name,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,rain_1h,clouds_all,weather_description
0,1630454400,2021-09-01 00:00:00 +0000 UTC,local,18.74,18.84,15.72,20.34,1015,,,83,1.18,,78,broken clouds
1,1630458000,2021-09-01 01:00:00 +0000 UTC,local,18.73,18.83,15.72,20.34,1014,,,83,1.46,,92,overcast clouds
2,1630461600,2021-09-01 02:00:00 +0000 UTC,local,17.84,17.98,16.72,20.34,1014,,,88,1.05,,91,overcast clouds
3,1630465200,2021-09-01 03:00:00 +0000 UTC,local,18.27,18.40,16.68,20.34,1014,,,86,0.46,0.14,94,light rain
4,1630468800,2021-09-01 04:00:00 +0000 UTC,local,17.81,17.97,16.12,20.34,1013,,,89,0.93,1.26,95,moderate rain
5,1630472400,2021-09-01 05:00:00 +0000 UTC,local,17.89,18.06,16.12,17.93,1014,,,89,1.10,0.96,94,light rain
6,1630476000,2021-09-01 06:00:00 +0000 UTC,local,18.27,18.48,16.12,18.48,1014,,,89,1.21,0.11,93,light rain
7,1630479600,2021-09-01 07:00:00 +0000 UTC,local,19.44,19.69,16.68,19.59,1015,,,86,1.27,4.55,95,heavy intensity rain
8,1630483200,2021-09-01 08:00:00 +0000 UTC,local,18.92,19.19,16.68,19.04,1015,,,89,2.33,3.38,99,moderate rain
9,1630486800,2021-09-01 09:00:00 +0000 UTC,local,18.98,19.28,17.23,19.04,1015,,,90,2.23,0.52,100,light rain


In [8]:
#Null values processing

In [9]:
#df_en.isna().any()
df_en.isna().any()

Data                       False
Hora                       False
Normal (kWh)               False
Horário Económico (kWh)    False
Autoconsumo (kWh)          False
Injeção na rede (kWh)      False
dtype: bool

In [10]:
df_en["Injeção na rede (kWh)"] = df_en["Injeção na rede (kWh)"].astype("category")
df_me["weather_description"] = df_me['weather_description'].astype("category")
df_me["city_name"] = df_me["city_name"].astype("category")

In [11]:
print(len(df_me))
df_me.isna().any()

11688


dt                     False
dt_iso                 False
city_name              False
temp                   False
feels_like             False
temp_min               False
temp_max               False
pressure               False
sea_level              False
grnd_level             False
humidity               False
wind_speed             False
rain_1h                False
clouds_all             False
weather_description    False
dtype: bool

In [20]:
#fix datetime in df_en
df_en['datetime'] = pd.to_datetime(df_en['Data'] + ' ' + df_en['Hora'].astype(str) + ':00:00')

In [32]:
#fix datetime in df_me (loses the UTC tag)
df_me['dt_iso'] = pd.to_datetime(df_me['dt_iso'], format='%Y-%m-%d %H:%M:%S %z UTC')
df_me['dt_iso'] = df_me['dt_iso'].dt.tz_localize(None)
df_me = df_me.rename(columns={"dt_iso": "datetime"})

In [40]:
df_me.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11688 entries, 0 to 11687
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   dt                   11688 non-null  int64         
 1   datetime             11688 non-null  datetime64[ns]
 2   city_name            11688 non-null  category      
 3   temp                 11688 non-null  float64       
 4   feels_like           11688 non-null  float64       
 5   temp_min             11688 non-null  float64       
 6   temp_max             11688 non-null  float64       
 7   pressure             11688 non-null  int64         
 8   sea_level            11688 non-null  object        
 9   grnd_level           11688 non-null  object        
 10  humidity             11688 non-null  int64         
 11  wind_speed           11688 non-null  float64       
 12  rain_1h              11688 non-null  object        
 13  clouds_all           11688 non-

In [42]:
outer_join_merged_df = pd.merge(df_en, df_me, on='datetime', how='outer')

In [43]:
inner_join_merged_df = pd.merge(df_en, df_me, on='datetime', how='inner')

In [45]:
outer_join_merged_df.isna().sum()

Data                       672
Hora                       672
Normal (kWh)               672
Horário Económico (kWh)    672
Autoconsumo (kWh)          672
Injeção na rede (kWh)      672
datetime                     0
dt                           0
city_name                    0
temp                         0
feels_like                   0
temp_min                     0
temp_max                     0
pressure                     0
sea_level                    0
grnd_level                   0
humidity                     0
wind_speed                   0
rain_1h                      0
clouds_all                   0
weather_description          0
dtype: int64

In [48]:
len(outer_join_merged_df)

11688

In [52]:
outer_join_merged_df.isna().sum()

Data                       672
Hora                       672
Normal (kWh)               672
Horário Económico (kWh)    672
Autoconsumo (kWh)          672
Injeção na rede (kWh)      672
datetime                     0
dt                           0
city_name                    0
temp                         0
feels_like                   0
temp_min                     0
temp_max                     0
pressure                     0
sea_level                    0
grnd_level                   0
humidity                     0
wind_speed                   0
rain_1h                      0
clouds_all                   0
weather_description          0
dtype: int64

In [ ]:
#at this point, if I do an inner join I am losing 5.7% of all datarows

In [51]:
#Sava the clean dataset
inner_join_merged_df.to_csv('~/Downloads/DAA/daasbstp2023/inner.csv')